## Import modules

In [ ]:
from Bio.Seq import Seq
from Bio import SeqIO
import doctest
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn import svm

## Generate combinations

In [ ]:
num_to_char = {
    0 : 'a',
    1 : 't',
    2 : 'c',
    3 : 'g'
}

char_to_num = {
    'a' : 0,
    't' : 1,
    'c' : 2,
    'g' : 3
}

combinations = set()

negative_data =[]
positive_data = []

FRAME_SIZE = 1500
STEP = 750

In [ ]:
def decode(n):
    '''
    Test cases:
    
    >>> decode(0)
    'aaaa'
    
    >>> decode(1)
    'taaa'
    
    >>> decode(2)
    'caaa'
    
    >>> decode(255)
    'gggg'
    '''
    ret = ''
    for i in range(0, 8, 2):
        tmp = n & ((1 << i) | (1 << (i+1)))
        tmp = tmp >> i
        ch = num_to_char[tmp]
        ret = ret + ch
    return ret

In [ ]:
doctest.testmod(verbose=False)

In [ ]:
def generate_combinations():
    for i in range(255):
        combination = decode(i)
        assert len(combination) == 4
        
        seq = Seq(combination)
        seq_rev = seq.reverse_complement()
        string = str(seq_rev)
        if (not(string in combinations)):
            combinations.add(combination)
            
    assert len(combinations) == 136
        

In [ ]:
generate_combinations()

## Read training data

In [ ]:
def read_data():
    

    for record in SeqIO.parse("vista1500", "fasta"):
        positive_data.append(str(record.seq).lower())

    for record in SeqIO.parse("randoms1500", "fasta"):
        negative_data.append(str(record.seq).lower())

In [ ]:
read_data()
positive_data

## Compute frequencies

In [ ]:
def process_sequence(seq):
    
    frequency_map = {}
    
    for combination in combinations:
        frequency_map[combination] = 0
        
    assert(len(frequency_map) == len(combinations))
    
    for i in range(len(seq) - 4):
        
        combination = seq[i:i+4]
        if (combination in frequency_map):
            frequency_map[combination] = frequency_map[combination] + 1
        
        combination_reverse_complement = str(Seq(combination).reverse_complement())
        
        if (combination_reverse_complement in frequency_map):
            frequency_map[combination_reverse_complement] = frequency_map[combination_reverse_complement] + 1
        
    ret_arr = []
    for key in frequency_map:
        
        ret_arr.append( float(float(frequency_map[key]) / 1500.0 ))
    return ret_arr
        

In [ ]:
def generate_training_data():
    
    x = []
    y = []
    
    for sq in positive_data:
        x.append(process_sequence(sq))
        y.append(1)
    for sq in negative_data:
        x.append(process_sequence(sq))
        y.append(0)
    return x, y

In [ ]:
x, y = generate_training_data()

### shuffle the arrays

In [ ]:
x, y = shuffle(x, y)

### split into train data and test data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4) # 50% training and 30% test

### create random forest classifyer

In [ ]:
clf=RandomForestClassifier(n_estimators=1000)

In [ ]:
clf.fit(x_train,y_train)

### evaluate the random forest classifyer

In [ ]:
y_test_pred=clf.predict(x_test)
y_train_pred=clf.predict(x_train)

In [ ]:
print("Test Accuracy:",metrics.accuracy_score(y_test, y_test_pred))
print("Train Accuracy:",metrics.accuracy_score(y_train, y_train_pred))

### create support vector machine classifyer

In [ ]:
clf = svm.SVC()

In [ ]:
clf.fit(x_train, y_train)

### evaluate support vector machine classifyer

In [ ]:
y_test_pred=clf.predict(x_test)

In [ ]:
print("Test Accuracy:",metrics.accuracy_score(y_test, y_test_pred))

## read chromosome

In [ ]:
def read_chr21():
    records = []
    for record in SeqIO.parse("chr21.fa", "fasta"):
        records.append(str(record.seq).lower())
    assert len(records) == 1
    return records[0]

In [ ]:
chr21 = read_chr21()

## count frequencies in chr21

In [ ]:
def process_chr(chr21):
    
    x = []
    contains_N = []
    for i in range(0, len(chr21) - FRAME_SIZE, STEP):
        sub_seq = chr21[i:i+FRAME_SIZE]
        
        assert len(sub_seq) == FRAME_SIZE
        if (sub_seq.find('n') == -1):
            x.append(process_sequence(sub_seq))
            contains_N.append(0)
        else:
            contains_N.append(1)
    
    predictions = clf.predict_proba(x)
    return predictions, contains_N


In [ ]:
predictions, contains_N = process_chr(chr21)

In [ ]:
def fill_means(predictions, contains_N):
    i = 0
    ret = []
    p = [prediction[1] for prediction in predictions]
    mean = float(sum(p) / len(p))
    for has_N in contains_N:
        
        if (has_N == 0):
            ret.append(p[i])
            i = i + 1
        else:
            ret.append(mean)
    return ret

In [ ]:
final_list = fill_means(predictions, contains_N)

## write to wig file

In [ ]:
with open("chr21.wig", "w") as f:
    f.write("fixedStep chrom=chr21 start=0 step=750 span=1500\n")
    f.write("\n".join([str(c) for c in final_list]))